In [ ]:
df = pd.read_csv("dataset/df_gpt3.csv")
df = pd.read_csv("dataset/df_LLM_feedback.csv")
df = pd.concat([df3, df4], axis=1)

In [ ]:
!pip3 -q install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
# Function to clean up the text
def clean_and_extract_justification(text):
    # Remove all newline characters
    text = text.replace('\n', ' ')
    # Find the start of "Justification" and remove everything before it
    justification_start = text.find("Justification")
    if justification_start != -1:
        return text[justification_start:]  # Keep text from "Justification" onward
    else:
        return text  # Return the original text if "Justification" is not found

# Apply the function to the 'LLM_feedback' column
df['Refined_LLM_feedback'] = df['LLM_feedback'].apply(clean_and_extract_justification)


# take feedback which doesn't pass the threshold and send to the LLM to generate refined 'future work' (iteraton 2)

In [ ]:
# now use the model to send the problem you found and generate 'refined' limitaions
import os
from openai import OpenAI

os.environ['OPENAI_API_KEY'] = ''

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)
generated_limiations = []
# abstracts = []
# Abstract	Introduction conclusion	Related work	methodology		experiment and results	extra
for i in range(len(df)):
    coherence_score = df['Coherence and Logic'][i]
    relevance_score = df['Relevance and Accuracy'][i]
    readability_score = df['Readability and Style'][i]
    grammar_score = df['Grammatical Correctness'][i]
    overall_score = df['Overall Impression'][i]
    bscore_f1 = df['BERTScore_F1'][i]

    if coherence_score > 3 and relevance_score > 3 and readability_score > 3 and grammar_score > 3 and overall_score > 3 and bscore_f1 > 0.85:
      generated_limiations.append([df['Future_Work'][i]])

    else:
      print("else",i)
      prompt_template = """You are an AI model trained to analyze scientific content and provide high-quality future research suggestions. Below, you will find sections
      from a scientific article, along with a justification for improving the "Future Work" section. Please use the article content and justification to generate an enhanced
      "Future Work" section that addresses the limitations mentioned.

      Your goal is to create a cohesive, logically structured, and engaging "Future Work" section that aligns closely with the article's content.
      The suggestion should also consider areas of improvement outlined in the justification, such as Coherence and Logic, Relevance and Accuracy, Readability and Style,
      Grammatical Correctness, and precision. Aim for a concise response within 100 words.

      Article Content:
      Abstract:
      "{abstract_text}"

      Introduction:
      "{introduction_text}"

      Conclusion:
      "{conclusion_text}"

      Justification:
      "{justification_text}"

      Based on the article content and the justification provided, generate an improved "Future Work" section that builds on the strengths of the original while addressing
      the noted weaknesses. Ensure your response is aligned with the main topic and presents clear directions for future research.
      """

      # Replace placeholders with the actual content from the DataFrame
      abstract_text = df.loc[i, 'Abstract']  # Replace 0 with the actual row index if needed
      introduction_text = df.loc[i, 'Introduction']
      conclusion_text = df.loc[i, 'Conclusion']
      justification_text = df.loc[i, 'Refined_LLM_feedback']

      # Format the final prompt
      prompt = prompt_template.format(
          abstract_text=abstract_text,
          introduction_text=introduction_text,
          conclusion_text=conclusion_text,
          justification_text=justification_text
      )

      summary_text = ""  # Initialize an empty string to collect the limitation text

      stream = client.chat.completions.create(
          model="gpt-3.5-turbo-1106", # gpt-3.5-turbo
          # model="gpt-4o-mini",
          messages=[
              {
                  "role": "user",
                  "content": prompt,
              }
          ],
          stream=True,
          temperature=0.2  # Adjust the temperature as needed, max_tokens=150
      )

      for chunk in stream:
          summary_chunk = chunk.choices[0].delta.content or ""
          summary_text += summary_chunk  # Append each chunk to the limitation_text

      summary_chunks = []
      summary_chunks.append(summary_text)
      generated_limiations.append(summary_chunks)  # Append the collected limitation text to the list

In [ ]:
df_generated_future_work = pd.DataFrame(generated_limiations, columns=['Future Work'])
df['Future_Work'] = df_generated_future_work['Future Work']

In [ ]:
!pip3 -q install bert-score

Evaluation

In [ ]:
import pandas as pd
from bert_score import BERTScorer

refs = df['GPT_ground_truth_FW'].apply(lambda x: [str(x).strip()]).tolist()
cands = df['Future_Work'].apply(lambda x: [str(x).strip()]).tolist()

scorer = BERTScorer(model_type='roberta-large')

precision_scores = []
recall_scores = []
f1_scores = []
total_precision, total_recall, total_f1 = 0, 0, 0

# Loop through each reference and candidate
for ref, cand in zip(refs, cands):
    if ref and cand:  # Ensure neither is empty
        # Calculate scores
        P, R, F1 = scorer.score(cand, ref)

        # Convert to scalar values for printing and storing
        precision = P.mean().item()
        recall = R.mean().item()
        f1_score = F1.mean().item()

        # Append scores to respective lists
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1_score)

        # Update total accumulators
        total_precision += precision
        total_recall += recall
        total_f1 += f1_score

        # Print each score along with refs and cands
        print(f"Precision: {precision}, Recall: {recall}, F1 Score: {f1_score}")
        print(f"Reference: {ref}")
        print(f"Candidate: {cand}\n")

# Calculate averages
num_scores = len(precision_scores)
average_precision = total_precision / num_scores if num_scores > 0 else 0
average_recall = total_recall / num_scores if num_scores > 0 else 0
average_f1 = total_f1 / num_scores if num_scores > 0 else 0

# Add the individual scores to the DataFrame
df['BERTScore_Precision'] = precision_scores
df['BERTScore_Recall'] = recall_scores
df['BERTScore_F1'] = f1_scores


In [ ]:
!pip3 -q install evaluate
!pip -q install rouge_score

In [ ]:
df = df.astype(str)

def lcs(X, Y):
    m = len(X)
    n = len(Y)
    L = [[0] * (n + 1) for i in range(m + 1)]

    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0:
                L[i][j] = 0
            elif X[i - 1] == Y[j - 1]:
                L[i][j] = L[i - 1][j - 1] + 1
            else:
                L[i][j] = max(L[i - 1][j], L[i][j - 1])
    return L[m][n]

def rough_1(generated_summary, reference_summary):
    gen_unigrams = set(generated_summary.split())
    ref_unigrams = set(reference_summary.split())

    common_unigrams = gen_unigrams.intersection(ref_unigrams)
    precision = len(common_unigrams) / len(gen_unigrams)
    return precision

def rough_2(generated_summary, reference_summary):
    gen_bigrams = set(zip(generated_summary.split(), generated_summary.split()[1:]))
    ref_bigrams = set(zip(reference_summary.split(), reference_summary.split()[1:]))
    common_bigrams = gen_bigrams.intersection(ref_bigrams)
    precision = len(common_bigrams) / len(gen_bigrams)
    return precision

total_rough2_score = 0
for i in range(len(df)):
  rough_2_score = rough_2(df['Future_Work'][i], df['GPT_ground_truth_FW'][i])
  total_rough2_score += rough_2_score
print("average_rough_2_score",total_rough2_score/len(df))


total_rough1_score = 0
for i in range(len(df)):
  rough_1_score = rough_1(df['Future_Work'][i], df['GPT_ground_truth_FW'][i])
  total_rough1_score += rough_1_score
print("average_rough_1_score",total_rough1_score/len(df))

# cosine similarity
def rough_l(generated_summary, reference_summary):
    lcs_length = lcs(generated_summary.split(), reference_summary.split())
    precision = lcs_length / len(reference_summary.split())
    recall = lcs_length / len(generated_summary.split())

    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score


total_rough_l_score = 0
for i in range(len(df)):
  rough_l_score = rough_l(df['Future_Work'][i], df['GPT_ground_truth_FW'][i])
  total_rough_l_score += rough_l_score
print("average_rough_l_score",total_rough_l_score/len(df))

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(text1, text2):
    # Use CountVectorizer to convert text to a matrix of token counts
    vectorizer = CountVectorizer().fit_transform([text1, text2])

    # Calculate cosine similarity between the two texts
    similarity_matrix = cosine_similarity(vectorizer)

    # The similarity_matrix[0, 1] contains the cosine similarity between text1 and text2
    return similarity_matrix[0, 1]

def calculate_average_rough_score(text1, text2):
    # Calculate similarity scores
    similarity_score1 = calculate_similarity(text1, text2)
    similarity_score2 = calculate_similarity(text2, text1)

    # Calculate average rough score
    average_rough_score = (similarity_score1 + similarity_score2) / 2

    return average_rough_score

total_score = 0  # Initialize a variable to accumulate scores

for i in range(len(df)):
    # reference_summary = df_grouped_sentence['Sentence'][i] test_data['generated_future_work'][i], test_data['Extracted Future Work'][i]
    reference_summary = df['GPT_ground_truth_FW'][i]
    generated_summary = df['Future_Work'][i]
    average_rough_score = calculate_average_rough_score(reference_summary, generated_summary)

    total_score += average_rough_score

# Calculate the overall average
overall_average = total_score / len(df)

# Print the overall average
print(f"\nOverall Average cosine similarity Score: {overall_average}")

import pandas as pd

def jaccard_similarity(text1, text2):
    # Tokenize the texts by splitting on whitespace
    set1 = set(text1.split())
    set2 = set(text2.split())

    # Calculate the intersection and union of the sets
    intersection = set1.intersection(set2)
    union = set1.union(set2)

    # Compute Jaccard similarity
    return len(intersection) / len(union) if union else 0

# Calculate Jaccard similarity for each pair of rows test_data['Extracted Future Work'][i]
jaccard_similarities = []

for i in range(len(df)):
    similarity = jaccard_similarity(df.loc[i, 'GPT_ground_truth_FW'], df.loc[i, 'Future_Work'])
    jaccard_similarities.append(similarity)

# Add the Jaccard similarity as a new column in df1
df['Jaccard_Similarity'] = jaccard_similarities

# Calculate the average Jaccard similarity
average_jaccard_similarity = sum(jaccard_similarities) / len(jaccard_similarities)

print("average jaccard similarity",average_jaccard_similarity)

!pip install nltk
import nltk
nltk.download('punkt')
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from nltk import word_tokenize
import numpy as np

def calculate_bleu(reference, candidate):
    # Tokenize the sentences using nltk's word_tokenize function
    reference_tokens = word_tokenize(reference.lower())
    candidate_tokens = word_tokenize(candidate.lower())

    # Calculate BLEU score
    score = sentence_bleu([reference_tokens], candidate_tokens)
    return score

# First, ensure that both columns are converted to string type and handle any missing values
df['GPT_ground_truth_FW'] = df['GPT_ground_truth_FW'].astype(str)
df['Future_Work'] = df['Future_Work'].astype(str)

# Calculate BLEU scores using a list comprehension and store in a list
bleu_scores = [calculate_bleu(ref, cand) for ref, cand in zip(df['GPT_ground_truth_FW'],df['Future_Work'])]

# Optional: Store the BLEU scores back into a DataFrame for analysis
results_df = pd.DataFrame({'BLEU Score': bleu_scores})

# Calculate the average BLEU score
average_bleu_score = results_df['BLEU Score'].mean()

print(f"Average BLEU Score: {average_bleu_score}")

In [ ]:
!pip install -q groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


evaluation by LLM

In [ ]:
from google.colab import userdata
prompt1 = """Title: Evaluation of Text Quality

Instructions:

You are provided with two texts for each pair: one is generated by a machine (Machine-Generated Text), and the other is the original or ground truth text (Ground Truth). Please read both texts carefully. After reviewing each text, assign a score from 1 to 5 based on the criteria outlined below. The score should reflect how well the machine-generated text compares to the ground truth, where 1 represents poor quality and 5 represents excellent quality that closely matches or even surpasses the ground truth in some aspects.

Scoring Criteria:
Coherence and Logic:

5: The text is exceptionally coherent; the ideas flow logically and are well connected.
3: The text is coherent but may have occasional lapses in logic or flow.
1: The text is disjointed or frequently illogical.
Relevance and Accuracy:

5: The text is completely relevant to the topic and accurate in all presented facts.
3: The text is generally relevant with minor factual errors or slight deviations from the topic.
1: The text often strays off topic or includes multiple factual inaccuracies.
Readability and Style:

5: The text is engaging, well-written, and stylistically consistent with the ground truth.
3: The text is readable but may lack flair or have minor stylistic inconsistencies.
1: The text is difficult to read or stylistically poor.
Grammatical Correctness:

5: The text is free from grammatical errors.
3: The text has occasional grammatical errors that do not impede understanding.
1: The text has frequent grammatical errors that hinder comprehension.
Overall Impression:

5: The text is of a quality that you would expect from a professional writer.
3: The text is acceptable but would benefit from further editing.
1: The text is of a quality that needs significant revision to be usable.
Task:

For each text pair:

Rate the Machine-Generated Text on each criterion and provide a final overall score out of 5.
Provide a brief justification for your scores, highlighting strengths and weaknesses observed in the machine-generated text relative to the ground truth.
Example of Usage:
Text Pair 1:

Machine-Generated Text: "The quick brown fox jumps over the lazy dog repeatedly."

Ground Truth: "A quick brown fox consistently jumps over the lazy dog."

Evaluation:

Coherence and Logic: 5
Relevance and Accuracy: 4
Readability and Style: 5
Grammatical Correctness: 5
Overall Impression: 5
Justification: The machine-generated text maintains the core message and style of the ground truth, presenting it coherently and engagingly. Minor variations in wording do not impact the overall quality or relevance of the message. """

In [ ]:
import os

from groq import Groq

client = Groq(
    api_key=userdata.get('GROQ_API_KEY'),
)

In [ ]:
import os
from openai import OpenAI

os.environ['OPENAI_API_KEY'] = ''
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)
LLM_feedback = []

for i in range(len(df)):
    prompt2 = f"Ground Truth: {df['GPT_ground_truth_FW'][i]} Machine Generate Text: {df['Future_Work'][i]}"
    prompt = prompt1 + prompt2
    summary_text = ""  # Initialize an empty string to collect the limitation text

    stream = client.chat.completions.create(
        model="gpt-3.5-turbo",
        # model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        stream=True,
        temperature=0.2
    )
    for chunk in stream:
        feedback_chunk = chunk.choices[0].delta.content or ""
        feedback_text += feedback_chunk

    feedback_chunks = []
    feedback_chunks.append(feedback_text)
    LLM_feedback.append(feedback_chunks)

In [ ]:
df_LLM_feedback = pd.DataFrame(generated_summary, columns=['LLM_feedback'])

In [ ]:
import pandas as pd
import numpy as np
import re

# Function to extract scores from each category
def extract_score(text, category):
    pattern = rf"{category}: (\d)"
    score = re.search(pattern, text)
    return int(score.group(1)) if score else np.nan

# Apply the function for each category
df_LLM_feedback['Coherence and Logic'] = df_LLM_feedback['LLM_feedback'].apply(extract_score, category='Coherence and Logic')
df_LLM_feedback['Relevance and Accuracy'] = df_LLM_feedback['LLM_feedback'].apply(extract_score, category='Relevance and Accuracy')
df_LLM_feedback['Readability and Style'] = df_LLM_feedback['LLM_feedback'].apply(extract_score, category='Readability and Style')
df_LLM_feedback['Grammatical Correctness'] = df_LLM_feedback['LLM_feedback'].apply(extract_score, category='Grammatical Correctness')
df_LLM_feedback['Overall Impression'] = df_LLM_feedback['LLM_feedback'].apply(extract_score, category='Overall Impression')

# Calculate average for each category
average_coherence_logic = df_LLM_feedback['Coherence and Logic'].mean()
average_relevance_accuracy = df_LLM_feedback['Relevance and Accuracy'].mean()
average_readability_style = df_LLM_feedback['Readability and Style'].mean()
average_grammatical_correctness = df_LLM_feedback['Grammatical Correctness'].mean()
average_overall_impression = df_LLM_feedback['Overall Impression'].mean()

# take the feedback who doesn't pass the threshold and send to LLM (3rd iteration)

In [ ]:
df['BERTScore_F1'] = pd.to_numeric(df['BERTScore_F1'], errors='coerce')
df['Coherence and Logic'] = pd.to_numeric(df['Coherence and Logic'], errors='coerce')
df['Relevance and Accuracy'] = pd.to_numeric(df['Relevance and Accuracy'], errors='coerce')
df['Readability and Style'] = pd.to_numeric(df['Readability and Style'], errors='coerce')
df['Grammatical Correctness'] = pd.to_numeric(df['Grammatical Correctness'], errors='coerce')
df['Overall Impression'] = pd.to_numeric(df['Overall Impression'], errors='coerce')


In [ ]:
!pip3 -q install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
# Function to clean up the text
def clean_and_extract_justification(text):
    # Remove all newline characters
    text = text.replace('\n', ' ')
    # Find the start of "Justification" and remove everything before it
    justification_start = text.find("Justification")
    if justification_start != -1:
        return text[justification_start:]  # Keep text from "Justification" onward
    else:
        return text  # Return the original text if "Justification" is not found

# Apply the function to the 'LLM_feedback' column
df['Refined_LLM_feedback'] = df['LLM_feedback'].apply(clean_and_extract_justification)


In [ ]:
# now use the model to send the problem you found and generate 'refined' limitaions
import os
from openai import OpenAI

os.environ['OPENAI_API_KEY'] = ''

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)
generated_limiations = []
# abstracts = []
# Abstract	Introduction conclusion	Related work	methodology		experiment and results	extra
for i in range(len(df)):
    coherence_score = df['Coherence and Logic'][i]
    relevance_score = df['Relevance and Accuracy'][i]
    readability_score = df['Readability and Style'][i]
    grammar_score = df['Grammatical Correctness'][i]
    overall_score = df['Overall Impression'][i]
    bscore_f1 = df['BERTScore_F1'][i]

    if coherence_score > 3 and relevance_score > 3 and readability_score > 3 and grammar_score > 3 and overall_score > 3 and bscore_f1 > 0.86:
      generated_limiations.append([df['Future_Work'][i]])

    else:
      print("else",i)
      prompt_template = """You are an AI model trained to analyze scientific content and provide high-quality future research suggestions. Below, you will find sections
      from a scientific article, along with a justification for improving the "Future Work" section. Please use the article content and justification to generate an enhanced
      "Future Work" section that addresses the limitations mentioned.

      Your goal is to create a cohesive, logically structured, and engaging "Future Work" section that aligns closely with the article's content.
      The suggestion should also consider areas of improvement outlined in the justification, such as Coherence and Logic, Relevance and Accuracy, Readability and Style,
      Grammatical Correctness, and precision. Aim for a concise response within 100 words.

      Article Content:
      Abstract:
      "{abstract_text}"

      Introduction:
      "{introduction_text}"

      Conclusion:
      "{conclusion_text}"

      Justification:
      "{justification_text}"

      Based on the article content and the justification provided, generate an improved "Future Work" section that builds on the strengths of the original while addressing
      the noted weaknesses. Ensure your response is aligned with the main topic and presents clear directions for future research.
      """

      # Replace placeholders with the actual content from the DataFrame
      abstract_text = df.loc[i, 'Abstract']  # Replace 0 with the actual row index if needed
      introduction_text = df.loc[i, 'Introduction']
      conclusion_text = df.loc[i, 'Conclusion']
      justification_text = df.loc[i, 'Refined_LLM_feedback']

      # Format the final prompt
      prompt = prompt_template.format(
          abstract_text=abstract_text,
          introduction_text=introduction_text,
          conclusion_text=conclusion_text,
          justification_text=justification_text
      )

      summary_text = ""  # Initialize an empty string to collect the limitation text

      stream = client.chat.completions.create(
          model="gpt-3.5-turbo-1106", # gpt-3.5-turbo
          # model="gpt-4o-mini",
          messages=[
              {
                  "role": "user",
                  "content": prompt,
              }
          ],
          stream=True,
          temperature=0.2  # Adjust the temperature as needed, max_tokens=150
      )

      for chunk in stream:
          summary_chunk = chunk.choices[0].delta.content or ""
          summary_text += summary_chunk  # Append each chunk to the limitation_text

      summary_chunks = []
      summary_chunks.append(summary_text)
      generated_limiations.append(summary_chunks)  # Append the collected limitation text to the list

In [ ]:
df_generated_future_work = pd.DataFrame(generated_limiations, columns=['Future Work'])
df['Future_Work'] = df_generated_future_work['Future Work']

Evaluation

In [ ]:
!pip3 -q install bert-score

In [ ]:
import pandas as pd
from bert_score import BERTScorer

# Convert each 'Limitation' row to a list containing that single row's string
refs = df['GPT_ground_truth_FW'][:39].apply(lambda x: [str(x).strip()]).tolist()
cands = df['Future_Work'][:39].apply(lambda x: [str(x).strip()]).tolist()

# Initialize the scorer
scorer = BERTScorer(model_type='roberta-large')

# Initialize lists to store individual scores and accumulators for averages
precision_scores = []
recall_scores = []
f1_scores = []
total_precision, total_recall, total_f1 = 0, 0, 0

# Loop through each reference and candidate
for ref, cand in zip(refs, cands):
    if ref and cand:  # Ensure neither is empty
        # Calculate scores
        P, R, F1 = scorer.score(cand, ref)

        # Convert to scalar values for printing and storing
        precision = P.mean().item()
        recall = R.mean().item()
        f1_score = F1.mean().item()

        # Append scores to respective lists
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1_score)

        # Update total accumulators
        total_precision += precision
        total_recall += recall
        total_f1 += f1_score

        # Print each score along with refs and cands
        print(f"Precision: {precision}, Recall: {recall}, F1 Score: {f1_score}")
        print(f"Reference: {ref}")
        print(f"Candidate: {cand}\n")

# Calculate averages
num_scores = len(precision_scores)
average_precision = total_precision / num_scores if num_scores > 0 else 0
average_recall = total_recall / num_scores if num_scores > 0 else 0
average_f1 = total_f1 / num_scores if num_scores > 0 else 0

# Print average scores
print(f"Average Precision: {average_precision:.4f}")
print(f"Average Recall: {average_recall:.4f}")
print(f"Average F1 Score: {average_f1:.4f}")

# Add the individual scores to the DataFrame
df['BERTScore_Precision'][:39] = precision_scores
df['BERTScore_Recall'][:39] = recall_scores
df['BERTScore_F1'][:39] = f1_scores


In [ ]:
!pip3 -q install evaluate
!pip -q install rouge_score

In [ ]:
df = df.astype(str)

def lcs(X, Y):
    m = len(X)
    n = len(Y)
    L = [[0] * (n + 1) for i in range(m + 1)]

    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0:
                L[i][j] = 0
            elif X[i - 1] == Y[j - 1]:
                L[i][j] = L[i - 1][j - 1] + 1
            else:
                L[i][j] = max(L[i - 1][j], L[i][j - 1])
    return L[m][n]

def rough_1(generated_summary, reference_summary):
    gen_unigrams = set(generated_summary.split())
    ref_unigrams = set(reference_summary.split())

    common_unigrams = gen_unigrams.intersection(ref_unigrams)
    precision = len(common_unigrams) / len(gen_unigrams)
    return precision

def rough_2(generated_summary, reference_summary):
    gen_bigrams = set(zip(generated_summary.split(), generated_summary.split()[1:]))
    ref_bigrams = set(zip(reference_summary.split(), reference_summary.split()[1:]))
    common_bigrams = gen_bigrams.intersection(ref_bigrams)
    precision = len(common_bigrams) / len(gen_bigrams)
    return precision

total_rough2_score = 0
for i in range(len(df)):
  rough_2_score = rough_2(df['Future_Work'][i], df['GPT_ground_truth_FW'][i])
  total_rough2_score += rough_2_score
print("average_rough_2_score",total_rough2_score/len(df))


total_rough1_score = 0
for i in range(len(df)):
  rough_1_score = rough_1(df['Future_Work'][i], df['GPT_ground_truth_FW'][i])
  total_rough1_score += rough_1_score
print("average_rough_1_score",total_rough1_score/len(df))

# cosine similarity
def rough_l(generated_summary, reference_summary):
    lcs_length = lcs(generated_summary.split(), reference_summary.split())
    precision = lcs_length / len(reference_summary.split())
    recall = lcs_length / len(generated_summary.split())

    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

total_rough_l_score = 0
for i in range(len(df)):
  rough_l_score = rough_l(df['Future_Work'][i], df['GPT_ground_truth_FW'][i])
  total_rough_l_score += rough_l_score
print("average_rough_l_score",total_rough_l_score/len(df))

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(text1, text2):
    # Use CountVectorizer to convert text to a matrix of token counts
    vectorizer = CountVectorizer().fit_transform([text1, text2])

    # Calculate cosine similarity between the two texts
    similarity_matrix = cosine_similarity(vectorizer)

    # The similarity_matrix[0, 1] contains the cosine similarity between text1 and text2
    return similarity_matrix[0, 1]

def calculate_average_rough_score(text1, text2):
    # Calculate similarity scores
    similarity_score1 = calculate_similarity(text1, text2)
    similarity_score2 = calculate_similarity(text2, text1)

    # Calculate average rough score
    average_rough_score = (similarity_score1 + similarity_score2) / 2

    return average_rough_score

total_score = 0  # Initialize a variable to accumulate scores

for i in range(len(df)):
    # reference_summary = df_grouped_sentence['Sentence'][i] test_data['generated_future_work'][i], test_data['Extracted Future Work'][i]
    reference_summary = df['GPT_ground_truth_FW'][i]
    generated_summary = df['Future_Work'][i]
    average_rough_score = calculate_average_rough_score(reference_summary, generated_summary)

    # print(f"Similarity Score of Topic {i + 1}: {average_rough_score}")

    # Add the current score to the total
    total_score += average_rough_score

# Calculate the overall average
overall_average = total_score / len(df)

# Print the overall average
print(f"\nOverall Average cosine similarity Score: {overall_average}")

import pandas as pd

def jaccard_similarity(text1, text2):
    # Tokenize the texts by splitting on whitespace
    set1 = set(text1.split())
    set2 = set(text2.split())

    # Calculate the intersection and union of the sets
    intersection = set1.intersection(set2)
    union = set1.union(set2)

    # Compute Jaccard similarity
    return len(intersection) / len(union) if union else 0

# Calculate Jaccard similarity for each pair of rows test_data['Extracted Future Work'][i]
jaccard_similarities = []

for i in range(len(df)):
    similarity = jaccard_similarity(df.loc[i, 'GPT_ground_truth_FW'], df.loc[i, 'Future_Work'])
    jaccard_similarities.append(similarity)

# Add the Jaccard similarity as a new column in df1
df['Jaccard_Similarity'] = jaccard_similarities

# Calculate the average Jaccard similarity
average_jaccard_similarity = sum(jaccard_similarities) / len(jaccard_similarities)

print("average jaccard similarity",average_jaccard_similarity)

!pip install nltk
import nltk
nltk.download('punkt')
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
from nltk import word_tokenize
import numpy as np

def calculate_bleu(reference, candidate):
    # Tokenize the sentences using nltk's word_tokenize function
    reference_tokens = word_tokenize(reference.lower())
    candidate_tokens = word_tokenize(candidate.lower())

    # Calculate BLEU score
    score = sentence_bleu([reference_tokens], candidate_tokens)
    return score

# First, ensure that both columns are converted to string type and handle any missing values
df['GPT_ground_truth_FW'] = df['GPT_ground_truth_FW'].astype(str)
df['Future_Work'] = df['Future_Work'].astype(str)

# Calculate BLEU scores using a list comprehension and store in a list
bleu_scores = [calculate_bleu(ref, cand) for ref, cand in zip(df['GPT_ground_truth_FW'],df['Future_Work'])]

# Optional: Store the BLEU scores back into a DataFrame for analysis
results_df = pd.DataFrame({'BLEU Score': bleu_scores})

# Calculate the average BLEU score
average_bleu_score = results_df['BLEU Score'].mean()

print(f"Average BLEU Score: {average_bleu_score}")

In [ ]:
!pip install -q groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


evaluating by LLM

In [ ]:
from google.colab import userdata
prompt1 = """Title: Evaluation of Text Quality

Instructions:

You are provided with two texts for each pair: one is generated by a machine (Machine-Generated Text), and the other is the original or ground truth text (Ground Truth). Please read both texts carefully. After reviewing each text, assign a score from 1 to 5 based on the criteria outlined below. The score should reflect how well the machine-generated text compares to the ground truth, where 1 represents poor quality and 5 represents excellent quality that closely matches or even surpasses the ground truth in some aspects.

Scoring Criteria:
Coherence and Logic:

5: The text is exceptionally coherent; the ideas flow logically and are well connected.
3: The text is coherent but may have occasional lapses in logic or flow.
1: The text is disjointed or frequently illogical.
Relevance and Accuracy:

5: The text is completely relevant to the topic and accurate in all presented facts.
3: The text is generally relevant with minor factual errors or slight deviations from the topic.
1: The text often strays off topic or includes multiple factual inaccuracies.
Readability and Style:

5: The text is engaging, well-written, and stylistically consistent with the ground truth.
3: The text is readable but may lack flair or have minor stylistic inconsistencies.
1: The text is difficult to read or stylistically poor.
Grammatical Correctness:

5: The text is free from grammatical errors.
3: The text has occasional grammatical errors that do not impede understanding.
1: The text has frequent grammatical errors that hinder comprehension.
Overall Impression:

5: The text is of a quality that you would expect from a professional writer.
3: The text is acceptable but would benefit from further editing.
1: The text is of a quality that needs significant revision to be usable.
Task:

For each text pair:

Rate the Machine-Generated Text on each criterion and provide a final overall score out of 5.
Provide a brief justification for your scores, highlighting strengths and weaknesses observed in the machine-generated text relative to the ground truth.
Example of Usage:
Text Pair 1:

Machine-Generated Text: "The quick brown fox jumps over the lazy dog repeatedly."

Ground Truth: "A quick brown fox consistently jumps over the lazy dog."

Evaluation:

Coherence and Logic: 5
Relevance and Accuracy: 4
Readability and Style: 5
Grammatical Correctness: 5
Overall Impression: 5
Justification: The machine-generated text maintains the core message and style of the ground truth, presenting it coherently and engagingly. Minor variations in wording do not impact the overall quality or relevance of the message. """

In [ ]:
import os

from groq import Groq

client = Groq(
    api_key=userdata.get('GROQ_API_KEY'),
)

In [ ]:
import os
from openai import OpenAI

os.environ['OPENAI_API_KEY'] = ''
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)
LLM_feedback = []

for i in range(len(df)):
    prompt2 = f"Ground Truth: {df['GPT_ground_truth_FW'][i]} Machine Generate Text: {df['Future_Work'][i]}"
    prompt = prompt1 + prompt2
    summary_text = ""  # Initialize an empty string to collect the limitation text

    stream = client.chat.completions.create(
        model="gpt-3.5-turbo",
        # model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        stream=True,
        temperature=0.2
    )
    for chunk in stream:
        feedback_chunk = chunk.choices[0].delta.content or ""
        feedback_text += feedback_chunk

    feedback_chunks = []
    feedback_chunks.append(feedback_text)
    LLM_feedback.append(feedback_chunks)

In [ ]:
df_LLM_feedback = pd.DataFrame(generated_summary, columns=['LLM_feedback'])


In [ ]:
import pandas as pd
import numpy as np
import re

# Function to extract scores from each category
def extract_score(text, category):
    pattern = rf"{category}: (\d)"
    score = re.search(pattern, text)
    return int(score.group(1)) if score else np.nan

# Apply the function for each category
df_LLM_feedback['Coherence and Logic'] = df_LLM_feedback['LLM_feedback'].apply(extract_score, category='Coherence and Logic')
df_LLM_feedback['Relevance and Accuracy'] = df_LLM_feedback['LLM_feedback'].apply(extract_score, category='Relevance and Accuracy')
df_LLM_feedback['Readability and Style'] = df_LLM_feedback['LLM_feedback'].apply(extract_score, category='Readability and Style')
df_LLM_feedback['Grammatical Correctness'] = df_LLM_feedback['LLM_feedback'].apply(extract_score, category='Grammatical Correctness')
df_LLM_feedback['Overall Impression'] = df_LLM_feedback['LLM_feedback'].apply(extract_score, category='Overall Impression')

# Calculate average for each category
average_coherence_logic = df_LLM_feedback['Coherence and Logic'].mean()
average_relevance_accuracy = df_LLM_feedback['Relevance and Accuracy'].mean()
average_readability_style = df_LLM_feedback['Readability and Style'].mean()
average_grammatical_correctness = df_LLM_feedback['Grammatical Correctness'].mean()
average_overall_impression = df_LLM_feedback['Overall Impression'].mean()

In [ ]:
df.to_csv('dataset/df_LLM_feedback.csv', index=False)  # Set index=False if you don't want to save the row indices